# Convolutional Neural Network

### Importing the libraries

In [34]:
# 12/01/2024
# IDE: VSC
# Section 37: Convolutional Neural Networks (CNN)
# Video [355, 360]

# pip install tensorflow==2.14.0

import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [35]:
tf.__version__

'2.14.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [36]:
# we will apply some image transformations on the training set: avoid overfitting
# transformations on the images: zoom, rotations, shifting pixels

# https://keras.io/2.15/api/data_loading
# https://keras.io/2.15/api/data_loading/image/

train_datagen = ImageDataGenerator(

    # feature scaling to each pixel by dividing their value by 255
    # (feature scaling is absolutely compulsory for NNs)
    rescale=1./255,

    # preventing overfitting:
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

training_set = train_datagen.flow_from_directory(
    
    # training_set path
    'dataset/training_set/',

    # final size of the image
    target_size=(64, 64),
    # nr of images in each batch
    batch_size=32,
    # binary: cat or dog
    class_mode='binary'
)

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [37]:
# Remember when we were applying feature scaling to our training set and test set?
# We used the fit_transform method on the training set,
# but only the transform method on the test set
# In order to avoid information leakage
# Here it's exactly the same, we have to keep the images of the test set intact

test_datagen = ImageDataGenerator(
    # feature scaling to each pixel by dividing their value by 255
    # (feature scaling is absolutely compulsory for NNs)
    rescale=1./255
)

test_set = test_datagen.flow_from_directory(
    
    # test_set path
    'dataset/test_set/',

    # final size of the image
    target_size=(64, 64),
    # nr of images in each batch
    batch_size=32,
    # binary: cat or dog
    class_mode='binary'
)

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [38]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [39]:
# REMEMBER: Input Image (matrix) · Feature Detector (matrix) = Feature Map (matrix)

# filters: feature detectors
# kernel size: size of the feature detector (nº of rows (square matrix))
# activation: activation function ReLU (Rectified Linear Unit)
# input shape: only for the 1st layer
#   - colored images (64, 64, 3)
#   - black and white (64, 64, 1)

cnn.add(tf.keras.layers.Conv2D(
    filters=32,
    kernel_size=3,
    activation='relu',
    input_shape=[64, 64, 3]
))

### Step 2 - Pooling

In [40]:
# REMEMBER: Feature Map (matrix) --> Pooled Feature Map (matrix)

# pool_size: size of pooling matrix (you pick the max value out of the pool)
# strides: nr of pixels by which the pooling matrix is shifted to the right when we perform the pooling
# padding: when your pooling matrix is out of the edge

cnn.add(tf.keras.layers.MaxPool2D(
    pool_size=2,
    strides=2
))

### Adding a second convolutional layer

In [41]:
cnn.add(tf.keras.layers.Conv2D(
    filters=32,
    kernel_size=3,
    activation='relu'
))

cnn.add(tf.keras.layers.MaxPool2D(
    pool_size=2,
    strides=2
))

### Step 3 - Flattening

In [42]:
# flattening the results of all the convolutions and poolings into 1D vector
# that will become the input of a fully connected ANN

cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [43]:
# units=14 --> 14 neurons (Hyperparameter)
# activation='relu' --> re_ctifier l_inear u_nit --> rectifier activation function

# activation function:
# sigmoid: yes or no, 0 or 1
# softmax: classification with >2 categories or classes to predict
# no activation function: regression, continuous real number

# Adding the input layer and the first hidden layer
cnn.add(tf.keras.layers.Dense(units=14, activation='relu'))

### Step 5 - Output Layer

In [44]:
# units=1 --> only 1 output neuron is needed to encode a binary outcome
# activation='sigmoid' --> it allows to get a probability prediction
# activation='softmax' --> predicting more than a binary outcome

cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [45]:
# the best optimizers are the ones that can perform gradient descent
# stochastic gradient descent: it updates the weights to reduce the loss error between predictions and results

# non-binary loss function--> loss= 'categorical_crossentropy'
cnn.compile(optimizer= 'adam', loss= 'binary_crossentropy', metrics= ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [ ]:
# fit method --> will train the cnn on the training set
# x: data with which we train our model
# validation_data: evaluating it on the Test set
# epochs: training so that it converges, in this case minimum 25 

cnn.fit(x = training_set, validation_data=test_set, epochs=25)

# After 25 epochs:
# 1st accuracy: training_set --> 89%
# 2nd accuracy: test_set --> 80%

# if we hadn't done the image augmentation preprocessing in part 1
# we would have ended up with:
# 1st accuracy: training_set --> 99% --> overfitting
# 2nd accuracy: test_set --> 69%

## Part 4 - Making a single prediction

In [ ]:
# 1 Dog | 0 Cat
training_set.class_indices

{'cats': 0, 'dogs': 1}

In [47]:
import numpy as np
from keras.preprocessing import image

# [1,3]: dogs
# [4,6]: cats
# [7]: selfie

for i in range(1, 8):
    path_single_prediction = 'dataset/single_prediction/cat_or_dog_' +str(i) +'.jpg'
    test_image = image.load_img(path_single_prediction, target_size=(64,64))

    # remember when the input needed [[]], here is the same by converting it to a numpy array
    test_image = image.img_to_array(test_image)

    # The predict method has to be called into the exact same format that was used during the training
    # During the preprocessing we were training the CNN with batches of 32 images,
    # so this single image has to be fed as a batch
    # axis: the dimension of the batch is always the 1st dimension you give first the dimension and
    # then inside each batch you get the different images
    test_image = np.expand_dims(test_image, axis=0)

    result = cnn.predict(test_image)

    # {'cats': 0, 'dogs': 1}
    # training_set.class_indices

    # we access the 1st element in the batch result:
    if result[0][0] == 1:
        prediction = 'dog'
    else:
        prediction = 'cat'

    print('cat_or_dog_' +str(i) +'.jpg --> ', prediction)

1/1 [==============================] - 0s 60ms/step
cat_or_dog_1.jpg -->  dog
1/1 [==============================] - 0s 22ms/step
cat_or_dog_2.jpg -->  dog
1/1 [==============================] - 0s 21ms/step
cat_or_dog_3.jpg -->  dog
1/1 [==============================] - 0s 19ms/step
cat_or_dog_4.jpg -->  dog
1/1 [==============================] - 0s 20ms/step
cat_or_dog_5.jpg -->  dog
1/1 [==============================] - 0s 23ms/step
cat_or_dog_6.jpg -->  dog
1/1 [==============================] - 0s 21ms/step
cat_or_dog_7.jpg -->  dog
